# Case Study: A Fund Raising Net Return Prediction Model

## Background and objectives

"Healthcare for All", is a not-for-profit organization that provides financial help to people who are not able to afford healthcare. "Healthcare for All" raises funds through donations from all across the country (United States). They have an in-house database of over 13 million donors. "Healthcare for All" has been consistently raising money through various campaigns to request people to contribute to their cause. They reach out to the donors through various channels including personal mails, emails, fundraising events, reaching out to other businesses and corporations, and other philanthropists. One of their most efficient channels for a long period of time has been direct mails. But for the last couple of years, they have seen a decline in the donations through this medium.

You are working for "Healthcare for All" as an analyst. You will be analyzing the results of one of their recent direct mail fundraising appeals. This mailing was sent to a total of 3.5 million "Healthcare for All" donors who were on the database as of June 1997. Everyone included in this mailing had made at least one prior donation.

The mailing included a gift (or "premium") of personalized name & address labels plus an assortment of 10 note cards and envelopes. All of the donors who received this mailing were acquired by "Healthcare for All" through similar premium-oriented appeals such as this.

One group that is of particular interest to the organization is "Lapsed" donors. These are individuals who made their last donation 13 to 24 months ago. They represent an important group to the organization, since the longer someone goes without donating, the less likely they will be to give again. Therefore, the recapture of these former donors is a critical aspect of their fund raising efforts.

However, it was found that there is often an inverse correlation between likelihood to respond and the dollar amount of the gift, so a straight response model (a classification or discrimination task) will most likely net only very low dollar donors. High dollar donors will fall into the lower deciles, which would most likely be suppressed from future mailings. The lost revenue of these suppressed donors would then offset any gains due to the increased response rate of the low dollar donors.

Therefore, to improve the cost-effectiveness of future direct marketing efforts, "Healthcare for All" wishes to develop a model that will help them maximize the net revenue (a regression or estimation task) generated from future renewal mailings to Lapsed donors.

### Population

The population for this analysis will be Lapsed donors who received the June 1997 renewal mailing. Therefore, the analysis data set contains a subset of the total universe that received the mailing. The analysis file includes all 191,779 Lapsed donors who received the mailing, with responders to the mailing marked with a flag in the TARGET_B field. The total dollar amount of each responder's gift is in the TARGET_D field.

The overall response rate for this direct mail promotion is 5.1%. The distribution of the target fields in the learning and validation files is as follows:

Learning Data Set
Target Variable: Binary Indicator of Response to
Mailing

TARGET_B Frequency Percent

---

         0      90569      94.9
         1       4843       5.1

Target Variable: Donation Amount (in \$) to Mailing

Variable N Mean Minimum Maximum

---

TARGET_D 95412 0.7930732 0 200.0000000

---

Validation Data Set
Target Variable: Binary Indicator of Response to
Mailing

TARGET_B Frequency Percent

---

         0      91494      94.9
         1       4873       5.1

Target Variable: Donation Amount (in \$) to Mailing

Variable N Mean Minimum Maximum

---

TARGET_D 96367 0.7895819 0 500.0000000

---

### Cost Matrix

The package cost (including the mail cost) is \$0.68 per piece mailed.

### Analysis time frame and reference date

The mailing was sent out in June 2018. All information included in the file (excluding the giving history date fields) is reflective of behavior before 6/97. This date may be used as the reference date in generating the "number of months since" or "time since" or "elapsed time" variables. You can also find the reference date information in the filed `ADATE_2`. This field contains the dates the promotion was mailed.

## Data Sources and Order & Type of the variables in the data set

The dataset includes:

- 24 months of detailed promotion and giving history (covering the period 12 to 36 months before the mailing)
- A summary of the promotions sent to the donors over the most recent 12 months before the mailing (by definition, none of these donors responded to any of these promotions)
- Summary variables reflecting each donor's lifetime giving history (e.g., total # of donations before mailing, total \$ amount of the donations, etc.)
- Overlay demographics, including a mix of household and area level data
- All other available data from the database (e.g., date of first gift, state, origin source, etc.) 

##Which people will donate more?

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd

In [47]:
def load_original_data():
    file1 = pd.read_csv('Data/file1.csv')
    file2 = pd.read_csv('Data/file2.txt', sep = '\t')
    file3 = pd.read_excel('Data/file3.xlsx')
    file4 = pd.read_excel('Data/file4.xlsx')
    return pd.concat([file1,file2,file3, file4], axis=0)

In [48]:
hk_df =load_original_data()

In [50]:
hk_df

,CONTROLN,STATE,GENDER,HV1,IC1,IC4,HVP1,IC5,POBC1,POBC2,IC2,IC3,AVGGIFT,TCODE,DOB,DOMAIN,TARGET_D
0,44060,FL,M,AAA896,392,520.0,7,21975,6,16,430.0,466,28.000000,1,1901,C2,100.0
1,96093,IL,M,537.00,365,473.0,0,19387,1,89,415.0,410,5.666667,0,0,T2,7.0
2,43333,FL,F,725.00,301,436.0,3,18837,11,17,340.0,361,4.111111,0,2501,C2,5.0
3,21885,NC,M,AAA1095,401,413.0,7,14014,1,74,407.0,399,27.277778,0,2208,T2,38.0
4,190108,FL,F,995.00,252,348.0,0,17991,5,6,280.0,316,6.000000,28,0,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,41537,FL,F,742,396,470.0,0,15356,7,37,424.0,450,14.285714,2,6101,S2,50.0
1002,4881,FL,M,599,100,423.0,4,11670,10,27,406.0,200,9.230769,28,7401,C1,10.0
1003,66358,ND,M,593,258,299.0,0,10186,1,70,298.0,274,5.266667,1,3301,S2,5.0
1004,2231,WI,male,564,217,397.0,0,12315,0,96,386.0,295,11.400000,1,7301,T2,14.0


In [5]:
def lower_case_column_names(hk_df):
    hk_df.columns=[i.lower() for i in hk_df.columns] #LIST COMPREHENTION
    return hk_df

In [6]:
hk_df=lower_case_column_names(hk_df)

In [7]:
hk_df

,controln,state,gender,hv1,ic1,ic4,hvp1,ic5,pobc1,pobc2,ic2,ic3,avggift,tcode,dob,domain,target_d
0,44060,FL,M,AAA896,392,520.0,7,21975,6,16,430.0,466,28.000000,1,1901,C2,100.0
1,96093,IL,M,537.00,365,473.0,0,19387,1,89,415.0,410,5.666667,0,0,T2,7.0
2,43333,FL,F,725.00,301,436.0,3,18837,11,17,340.0,361,4.111111,0,2501,C2,5.0
3,21885,NC,M,AAA1095,401,413.0,7,14014,1,74,407.0,399,27.277778,0,2208,T2,38.0
4,190108,FL,F,995.00,252,348.0,0,17991,5,6,280.0,316,6.000000,28,0,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,41537,FL,F,742,396,470.0,0,15356,7,37,424.0,450,14.285714,2,6101,S2,50.0
1002,4881,FL,M,599,100,423.0,4,11670,10,27,406.0,200,9.230769,28,7401,C1,10.0
1003,66358,ND,M,593,258,299.0,0,10186,1,70,298.0,274,5.266667,1,3301,S2,5.0
1004,2231,WI,male,564,217,397.0,0,12315,0,96,386.0,295,11.400000,1,7301,T2,14.0


In [8]:
def rename_columns(hk_df):
    hk_df.rename(columns={'controln':'id','hv1':'median_home_val',
                          'ic1':'median_household_income',"ic2":"med_fam_income", 
                          "ic3":"avg_household_income","ic4": "avg_fam_income",
                          "ic5":"per_capita_income"}, inplace=True )
    return hk_df

In [9]:
hk_df=rename_columns(hk_df)

In [10]:
def drop_columns(hk_df) :
    hk_df.drop(columns=["id","tcode",'pobc1',"dob"], inplace=True)
    return hk_df

In [11]:
drop_columns(hk_df)

,state,gender,median_home_val,median_household_income,avg_fam_income,hvp1,per_capita_income,pobc2,med_fam_income,avg_household_income,avggift,domain,target_d
0,FL,M,AAA896,392,520.0,7,21975,16,430.0,466,28.000000,C2,100.0
1,IL,M,537.00,365,473.0,0,19387,89,415.0,410,5.666667,T2,7.0
2,FL,F,725.00,301,436.0,3,18837,17,340.0,361,4.111111,C2,5.0
3,NC,M,AAA1095,401,413.0,7,14014,74,407.0,399,27.277778,T2,38.0
4,FL,F,995.00,252,348.0,0,17991,6,280.0,316,6.000000,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,FL,F,742,396,470.0,0,15356,37,424.0,450,14.285714,S2,50.0
1002,FL,M,599,100,423.0,4,11670,27,406.0,200,9.230769,C1,10.0
1003,ND,M,593,258,299.0,0,10186,70,298.0,274,5.266667,S2,5.0
1004,WI,male,564,217,397.0,0,12315,96,386.0,295,11.400000,T2,14.0


In [12]:
list(hk_df.columns)

['state',
 'gender',
 'median_home_val',
 'median_household_income',
 'avg_fam_income',
 'hvp1',
 'per_capita_income',
 'pobc2',
 'med_fam_income',
 'avg_household_income',
 'avggift',
 'domain',
 'target_d']

In [13]:
hk_df = hk_df[[
 'state',
 "gender",
 'median_home_val',
 'avg_fam_income',
 'avg_household_income',
 'med_fam_income',
 'median_household_income',
 'hvp1',
 "per_capita_income",
 'pobc2',
 'avggift',
 'domain',
 'target_d']]

hk_df.head()

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,M,AAA896,520.0,466,430.0,392,7,21975,16,28.000000,C2,100.0
1,IL,M,537.00,473.0,410,415.0,365,0,19387,89,5.666667,T2,7.0
2,FL,F,725.00,436.0,361,340.0,301,3,18837,17,4.111111,C2,5.0
3,NC,M,AAA1095,413.0,399,407.0,401,7,14014,74,27.277778,T2,38.0
4,FL,F,995.00,348.0,316,280.0,252,0,17991,6,6.000000,C2,5.0


In [14]:
#NOW THE CLEANING BEGINS

In [15]:
hk_df.gender.unique() #it tells you which cathegories we have inside
hk_df.gender.value_counts()

F          1966
M          1474
male        128
female      106
Female       77
U            69
Male         33
J            23
feamale      15
A             1
Name: gender, dtype: int64

In [16]:
def clean_gender(x):
    if x in ['M', 'MALE',"male"]:
        return 'Male'
    elif x in ['F', 'feamale',"Female"]:
        return 'Female'
    elif np.nan:  pass
    else:
        return 'U'
    
hk_df['gender'] = list(map(clean_gender, hk_df['gender'])) 
hk_df

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,Male,AAA896,520.0,466,430.0,392,7,21975,16,28.000000,C2,100.0
1,IL,Male,537.00,473.0,410,415.0,365,0,19387,89,5.666667,T2,7.0
2,FL,Female,725.00,436.0,361,340.0,301,3,18837,17,4.111111,C2,5.0
3,NC,Male,AAA1095,413.0,399,407.0,401,7,14014,74,27.277778,T2,38.0
4,FL,Female,995.00,348.0,316,280.0,252,0,17991,6,6.000000,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,FL,Female,742,470.0,450,424.0,396,0,15356,37,14.285714,S2,50.0
1002,FL,Male,599,423.0,200,406.0,100,4,11670,27,9.230769,C1,10.0
1003,ND,Male,593,299.0,274,298.0,258,0,10186,70,5.266667,S2,5.0
1004,WI,Male,564,397.0,295,386.0,217,0,12315,96,11.400000,T2,14.0


In [17]:
hk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4028 entries, 0 to 1005
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   state                    4028 non-null   object 
 1   gender                   3660 non-null   object 
 2   median_home_val          4026 non-null   object 
 3   avg_fam_income           4027 non-null   float64
 4   avg_household_income     4028 non-null   int64  
 5   med_fam_income           4027 non-null   float64
 6   median_household_income  4028 non-null   int64  
 7   hvp1                     4028 non-null   int64  
 8   per_capita_income        4028 non-null   object 
 9   pobc2                    4028 non-null   int64  
 10  avggift                  4028 non-null   float64
 11  domain                   4028 non-null   object 
 12  target_d                 4028 non-null   float64
dtypes: float64(4), int64(4), object(5)
memory usage: 440.6+ KB


In [18]:
temp = hk_df.describe().transpose()
temp

,count,mean,std,min,25%,50%,75%,max
avg_fam_income,4027.0,444.664515,174.857272,0.00000,325.000000,410.00,528.000000,1331.0
avg_household_income,4028.0,399.788232,164.702061,0.00000,287.000000,365.00,479.250000,1311.0
med_fam_income,4027.0,400.639930,180.377938,0.00000,284.000000,366.00,477.000000,1500.0
median_household_income,4028.0,351.569265,168.316241,0.00000,239.000000,318.00,425.000000,1500.0
hvp1,4028.0,15.961519,28.841221,0.00000,0.000000,1.00,14.000000,99.0
pobc2,4028.0,55.476167,21.510554,0.00000,41.000000,57.00,73.000000,99.0
avggift,4028.0,11.771435,10.136313,2.26087,7.181818,10.00,14.388462,450.0
target_d,4028.0,15.645603,12.672374,1.00000,10.000000,13.46,20.000000,200.0


In [20]:
hk_df['median_home_val'] =  pd.to_numeric(hk_df['median_home_val'], errors='coerce')
hk_df['median_household_income'] =  pd.to_numeric(hk_df['median_household_income'], errors='coerce')
hk_df['per_capita_income'] =  pd.to_numeric(hk_df['per_capita_income'], errors='coerce')
hk_df

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,Male,NaN,520.0,466,430.0,392,7,21975.0,16,28.000000,C2,100.0
1,IL,Male,537.0,473.0,410,415.0,365,0,19387.0,89,5.666667,T2,7.0
2,FL,Female,725.0,436.0,361,340.0,301,3,18837.0,17,4.111111,C2,5.0
3,NC,Male,NaN,413.0,399,407.0,401,7,14014.0,74,27.277778,T2,38.0
4,FL,Female,995.0,348.0,316,280.0,252,0,17991.0,6,6.000000,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,FL,Female,742.0,470.0,450,424.0,396,0,15356.0,37,14.285714,S2,50.0
1002,FL,Male,599.0,423.0,200,406.0,100,4,11670.0,27,9.230769,C1,10.0
1003,ND,Male,593.0,299.0,274,298.0,258,0,10186.0,70,5.266667,S2,5.0
1004,WI,Male,564.0,397.0,295,386.0,217,0,12315.0,96,11.400000,T2,14.0


In [21]:
corrupted_rows_indices=list(hk_df[hk_df['median_home_val'].isnull()].index)

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,Male,NaN,520.0,466,430.0,392,7,21975.0,16,28.000000,C2,100.0
3,NC,Male,NaN,413.0,399,407.0,401,7,14014.0,74,27.277778,T2,38.0
8,CA,Female,NaN,250.0,235,206.0,184,0,8708.0,63,8.818182,T2,10.0
9,CA,Female,NaN,617.0,619,NaN,593,61,17838.0,62,6.666667,S1,10.0
35,TX,Male,NaN,531.0,450,457.0,368,5,33797.0,42,12.250000,U1,20.0
0,FL,Male,NaN,520.0,466,430.0,392,7,21975.0,16,28.000000,C2,100.0
5,IL,Male,764.0,501.0,480,477.0,457,1,16022.0,75,25.571429,S2,30.0
14,KS,Female,376.0,316.0,287,303.0,263,0,NaN,75,10.550000,T2,10.0
174,IN,Male,610.0,407.0,384,336.0,319,0,11815.0,86,4.625000,R2,7.0
201,FL,Female,1173.0,494.0,407,424.0,322,5,18738.0,14,4.125000,C1,3.0


In [26]:
corrupted_rows_indices
hk_df.iloc[corrupted_rows_indices,:]

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,Male,NaN,520.0,466,430.0,392,7,21975.0,16,28,C2,100.0
3,NC,Male,NaN,413.0,399,407.0,401,7,14014.0,74,27,T2,38.0
8,CA,Female,NaN,250.0,235,206.0,184,0,8708.0,63,8,T2,10.0
9,CA,Female,NaN,617.0,619,NaN,593,61,17838.0,62,6,S1,10.0
35,TX,Male,NaN,531.0,450,457.0,368,5,33797.0,42,12,U1,20.0
0,FL,Male,NaN,520.0,466,430.0,392,7,21975.0,16,28,C2,100.0
5,IL,Male,764.0,501.0,480,477.0,457,1,16022.0,75,25,S2,30.0
14,KS,Female,376.0,316.0,287,303.0,263,0,NaN,75,10,T2,10.0
174,IN,Male,610.0,407.0,384,336.0,319,0,11815.0,86,4,R2,7.0
201,FL,Female,1173.0,494.0,407,424.0,322,5,18738.0,14,4,C1,3.0


In [29]:
corrupted_rows_indices = list(hk_df[hk_df['median_home_val'].isnull()].index)
corrupted_rows_indices #gives you the numbers of rows in which there's NULL

[0, 3, 8, 9, 35, 0, 5, 14, 174, 201]

In [22]:
hk_df['avggift'] = hk_df['avggift'].astype('int')

In [23]:
hk_df.select_dtypes('object')

,state,gender,domain
0,FL,Male,C2
1,IL,Male,T2
2,FL,Female,C2
3,NC,Male,T2
4,FL,Female,C2
...,...,...,...
1001,FL,Female,S2
1002,FL,Male,C1
1003,ND,Male,S2
1004,WI,Male,T2


In [24]:
hk_df.select_dtypes('float64')

,median_home_val,avg_fam_income,med_fam_income,per_capita_income,target_d
0,NaN,520.0,430.0,21975.0,100.0
1,537.0,473.0,415.0,19387.0,7.0
2,725.0,436.0,340.0,18837.0,5.0
3,NaN,413.0,407.0,14014.0,38.0
4,995.0,348.0,280.0,17991.0,5.0
...,...,...,...,...,...
1001,742.0,470.0,424.0,15356.0,50.0
1002,599.0,423.0,406.0,11670.0,10.0
1003,593.0,299.0,298.0,10186.0,5.0
1004,564.0,397.0,386.0,12315.0,14.0


In [30]:
hk_df.drop_duplicates()

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,Male,NaN,520.0,466,430.0,392,7,21975.0,16,28,C2,100.0
1,IL,Male,537.0,473.0,410,415.0,365,0,19387.0,89,5,T2,7.0
2,FL,Female,725.0,436.0,361,340.0,301,3,18837.0,17,4,C2,5.0
3,NC,Male,NaN,413.0,399,407.0,401,7,14014.0,74,27,T2,38.0
4,FL,Female,995.0,348.0,316,280.0,252,0,17991.0,6,6,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,MI,Female,632.0,388.0,339,336.0,279,2,12653.0,71,8,0,5.0
997,FL,Male,595.0,274.0,262,263.0,252,0,11132.0,11,14,T2,20.0
998,CA,Female,2707.0,537.0,538,504.0,507,80,16165.0,54,12,U1,22.0
999,CA,Male,2666.0,653.0,612,609.0,535,63,24745.0,45,12,S1,21.0


In [33]:
nulls_df = pd.DataFrame(round(hk_df.isna().sum()/len(hk_df),4)*100)
hk_df.isna().sum()

state                        0
gender                     368
median_home_val             10
avg_fam_income               1
avg_household_income         0
med_fam_income               1
median_household_income      0
hvp1                         0
per_capita_income            6
pobc2                        0
avggift                      0
domain                       0
target_d                     0
dtype: int64

In [34]:
nulls_df = nulls_df.reset_index()

In [35]:
nulls_df.columns = ['header_name', 'percent_nulls']
nulls_df

,header_name,percent_nulls
0,state,0.00
1,gender,9.14
2,median_home_val,0.25
3,avg_fam_income,0.02
4,avg_household_income,0.00
5,med_fam_income,0.02
6,median_household_income,0.00
7,hvp1,0.00
8,per_capita_income,0.15
9,pobc2,0.00


In [32]:
columns_drop = nulls_df[nulls_df['percent_nulls']>1]['header_name']  # dummy case with 3 
print(columns_drop.values)

['gender']


In [36]:
hk_df[hk_df['gender'].isna()==True] # checking rows that are null based on a specific column 


,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
6,NC,None,775.0,419.0,380,364.0,318,5,13491.0,73,4,R2,12.0
13,AP,None,0.0,0.0,0,0.0,0,0,0.0,0,5,0,6.0
15,MI,None,890.0,523.0,512,474.0,463,1,9493.0,53,11,S1,15.0
19,CA,None,1951.0,625.0,619,578.0,556,44,17208.0,66,9,S1,12.0
21,OK,None,1040.0,656.0,579,609.0,472,0,26962.0,56,11,C2,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,MI,None,928.0,527.0,403,463.0,334,1,23732.0,57,15,S3,25.0
960,AL,None,367.0,276.0,236,220.0,179,0,9389.0,79,11,R3,20.0
966,FL,None,599.0,288.0,253,236.0,200,2,11547.0,34,7,T2,5.0
971,TX,None,1101.0,640.0,609,600.0,538,0,19641.0,48,16,S1,25.0


In [37]:
hk_df = hk_df[hk_df['med_fam_income'].isna()==False] # Since these nulls are not a lot, we can filter them 
hk_df = hk_df[hk_df['avg_fam_income'].isna()==False]
hk_df = hk_df[hk_df['per_capita_income'].isna()==False]

In [45]:
mean_median_home_value = np.mean(hk_df['median_home_val'])
hk_df['median_home_val'] = hk_df['median_home_val'].fillna(mean_median_home_value) #It's replacing the column

In [39]:
hk_df['gender'].value_counts()

Female    2050
Male      1602
Name: gender, dtype: int64

In [40]:
len(hk_df[hk_df['gender'].isna()==True])  # number of missing values

368

In [41]:
hk_df['gender'] = hk_df['gender'].fillna('F')

In [42]:
hk_df['gender'].unique() # check the unique values in the column

array(['Male', 'Female', 'F'], dtype=object)

In [43]:
pd.cut(hk_df['med_fam_income'],4) # to check the bins

0       (375.0, 750.0]
1       (375.0, 750.0]
2        (-1.5, 375.0]
3       (375.0, 750.0]
4        (-1.5, 375.0]
             ...      
1001    (375.0, 750.0]
1002    (375.0, 750.0]
1003     (-1.5, 375.0]
1004    (375.0, 750.0]
1005    (375.0, 750.0]
Name: med_fam_income, Length: 4020, dtype: category
Categories (4, interval[float64]): [(-1.5, 375.0] < (375.0, 750.0] < (750.0, 1125.0] < (1125.0, 1500.0]]

In [44]:
med_fam_income_labels = ['Low', 'Moderate', 'High', 'Very High']
hk_df['med_fam_income_binned'] = pd.cut(hk_df['med_fam_income'],4, labels=med_fam_income_labels)
hk_df

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d,med_fam_income_binned
0,FL,Male,1159.148591,520.0,466,430.0,392,7,21975.0,16,28,C2,100.0,Moderate
1,IL,Male,537.000000,473.0,410,415.0,365,0,19387.0,89,5,T2,7.0,Moderate
2,FL,Female,725.000000,436.0,361,340.0,301,3,18837.0,17,4,C2,5.0,Low
3,NC,Male,1159.148591,413.0,399,407.0,401,7,14014.0,74,27,T2,38.0,Moderate
4,FL,Female,995.000000,348.0,316,280.0,252,0,17991.0,6,6,C2,5.0,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,FL,Female,742.000000,470.0,450,424.0,396,0,15356.0,37,14,S2,50.0,Moderate
1002,FL,Male,599.000000,423.0,200,406.0,100,4,11670.0,27,9,C1,10.0,Moderate
1003,ND,Male,593.000000,299.0,274,298.0,258,0,10186.0,70,5,S2,5.0,Low
1004,WI,Male,564.000000,397.0,295,386.0,217,0,12315.0,96,11,T2,14.0,Moderate
